# CRF+LSTM

keras 2.2.4

tensorflow 1.13

pip install git+https://www.github.com/keras-team/keras-contrib.git

In [1]:
import re
import os

In [2]:
char_vocab_path = "CRF/data/char_vocabs.txt" # 字典文件
#train_data_path = 'data/train_data/train_data_000' # 训练数据
#train_data_path = './data/train_data' # 训练数据
#test_data_path = 'data/train_data/train_data_000' # 测试数据

special_words = ['<PAD>', '<UNK>'] # 特殊词表示

# "BIO"标记的标签
#label2idx = {"O": 0,
#             "B-PER": 1, "I-PER": 2,
#             "B-LOC": 3, "I-LOC": 4,
#             "B-ORG": 5, "I-ORG": 6
#            }
label2idx = {'O': 0,
             'B-DISEASE': 1, 'B-DISEASE_GROUP': 2,
             'B-DRUG_DOSAGE': 3, 'B-DRUG_EFFICACY': 4,
             'B-DRUG_INGREDIENT': 5, 'B-DRUG_TASTE': 6,
             'B-FOOD_GROUP':7, 'B-PERSON_GROUP':8,
             'B-SYMPTOM':9, 'B-SYNDROME':10,
             'I-DISEASE': 11, 'I-DISEASE_GROUP': 12,
             'I-DRUG_DOSAGE': 13, 'I-DRUG_EFFICACY': 14,
             'I-DRUG_INGREDIENT': 15, 'I-DRUG_TASTE': 16,
             'I-FOOD_GROUP':17, 'I-PERSON_GROUP':18,
             'I-SYMPTOM':19, 'I-SYNDROME':20
            }

# 索引和BIO标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

# 读取字符词典文件
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

In [3]:
# 读取训练语料
def read_corpus(corpus_path, vocab2idx, label2idx):
    with open(corpus_path, encoding='utf-8') as fr:
        lines = fr.readlines()

    sent_, tag_ = [], []
    for letter in lines:
        [char,label,_] = re.split('\t|\n',letter)
        sent_.append(char)
        tag_.append(label)

    sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent_]
    tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag_]
    return sent_ids, tag_ids

# 加载训练集
#train_datas, train_labels = read_corpus(train_data_path, vocab2idx, label2idx)
# 加载测试集
#test_datas, test_labels = read_corpus(test_data_path, vocab2idx, label2idx)


In [4]:
train_datas = []
train_labels = []
files = os.listdir('data/train_data')
for file in files:
    train_data_path_i = 'data/train_data/'+file
    train_datas_i, train_labels_i = read_corpus(train_data_path_i, vocab2idx, label2idx)
    train_datas.append(train_datas_i)
    train_labels.append(train_labels_i)
    #if i%10==0:
    #    print(i)

In [5]:
valid_datas = []
valid_labels = []
files = os.listdir('data/valid_data')
for file in files:
    valid_data_path_i = 'data/valid_data/'+file
    valid_datas_i, valid_labels_i = read_corpus(valid_data_path_i, vocab2idx, label2idx)
    valid_datas.append(valid_datas_i)
    valid_labels.append(valid_labels_i)

In [6]:
print(train_datas[50])
print([idx2vocab[idx] for idx in train_datas[50]])
print(train_labels[50])
print([idx2label[idx] for idx in train_labels[50]])

[0, 2659, 998, 2572, 211, 4166, 1412, 4167, 223, 3769, 2518, 4955, 182, 4955, 3682, 4033, 4211, 5428, 2558, 6802, 2659, 998, 589, 2644, 5483, 4843, 315, 398, 596, 659, 3102, 6802, 2471, 1083, 2838, 3556, 4127, 4471, 718, 4719, 6802, 2217, 680, 1534, 1588, 596, 4773, 5790, 6009, 885, 2157, 966, 6005, 2473, 694, 666, 4761, 4392, 34, 16, 60, 6802, 2471, 1083, 2053, 4622, 5253, 3212, 1950, 3647, 3903, 406, 3769, 182, 0, 4471, 806, 3765, 5940, 2288, 1575, 766, 308, 181, 217, 3186, 181, 794, 308, 4277, 2644, 585, 776, 6275, 236, 2773, 245, 1837, 5442, 6541, 6802, 1622, 3853, 4471, 181, 2643, 4471, 219, 5483, 2644, 2477, 3630, 659, 677, 249, 23, 16, 13, 17, 4, 181, 22, 20, 13, 21, 4, 6802, 3817, 2477, 2579, 5025, 6802, 1566, 577, 868, 6363, 6802, 3238, 850, 1401, 2002, 2938, 5793, 966, 466, 5608, 182, 0, 0, 3263, 1874, 3025, 617, 2646, 11, 211, 2937, 211, 5285, 11, 211, 2535, 16, 2937, 6817, 288, 2643, 4471, 694, 211, 950, 1874, 1438, 11, 2286, 4495, 4792, 2643, 4471, 2684, 17, 1368, 891, 515

In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking, Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Activation
from keras.preprocessing import sequence
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras import backend as K
K.clear_session()

EPOCHS = 15
BATCH_SIZE = 100
EMBED_DIM = 32
HIDDEN_SIZE = 16
MAX_LEN = 500
VOCAB_SIZE = len(vocab2idx)
CLASS_NUMS = len(label2idx)
print(VOCAB_SIZE, CLASS_NUMS)

print('padding sequences')
train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
valid_datas = sequence.pad_sequences(valid_datas, maxlen=MAX_LEN)
valid_labels = sequence.pad_sequences(valid_labels, maxlen=MAX_LEN)
print('x_train shape:', train_datas.shape)
print('x_test shape:', valid_datas.shape)

train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)
valid_labels = keras.utils.to_categorical(valid_labels, CLASS_NUMS)
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', valid_labels.shape)

## BiLSTM+CRF模型构建
inputs = Input(shape=(MAX_LEN,), dtype='int32')
x = Masking(mask_value=0)(inputs)
x = Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(x)
x = Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True))(x)
x = TimeDistributed(Dense(CLASS_NUMS))(x)
outputs = CRF(CLASS_NUMS)(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

model.compile(loss=crf_loss, optimizer='adam', metrics=[crf_viterbi_accuracy])
model.fit(train_datas, train_labels, epochs=EPOCHS, verbose=1, validation_split=0.1)

score = model.evaluate(valid_datas, valid_labels, batch_size=BATCH_SIZE)
print(model.metrics_names)
print(score)

Using TensorFlow backend.





6874 21
padding sequences
x_train shape: (901, 500)
x_test shape: (199, 500)
trainlabels shape: (901, 500, 21)
testlabels shape: (199, 500, 21)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500)               0         
_________________________________________________________________
masking_1 (Masking)          (None, 500)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 500, 32)           219968    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 500, 32)           6272      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 500, 21)           693       
_______________________________

In [9]:
# save model
model.save("model/ch_ner_model.h5")

In [10]:
from keras.models import load_model
import numpy as np

maxlen = 500
sentence = " 北京同仁堂科技发展股份有限公司制药厂  1.忌食辛辣，少进油腻。 2.感冒发热病人不宜服用。 3.有高血压、心脏病、肝病、糖尿病、肾病等慢性病严重者应在医师指导下服用。 4.伴有月经紊乱者，应在医师指导下服用。 5.眩晕症状较重者，应及时去医院就诊。 6.服药2周症状无缓解，应去医院就诊。 7.对本品过敏者禁用，过敏体质者慎用。 8.本品性状发生改变时禁止使用。 9.请将本品放在儿童不能接触的地方。 10.如正在使用其他药品，使用本品前请咨询医师或药师。  本品为浅黄色至棕黄色颗粒，气微香，味微苦。  滋养肝肾、宁心安神。用于更年期综合症属阴虚肝旺症，症见烘热汗出，头晕耳鸣，失眠多梦，五心烦热，腰背酸痛，大便干燥，心烦易怒，舌红少苔，脉弦细或弦细 开水冲服。一次1袋(12g)，一日3次。  如与其他药物同时使用可能会发生药物相互作用，详情请咨询医师或药师。  12g*10袋/盒  用于更年期综合症属阴虚肝旺症  铝塑复合膜包装，每袋装12克，每盒装10袋。  非处方药物（甲类）,中药保护品种二级  12g*10袋/盒  用于更年期综合症属阴虚肝旺更年期综合症气微香，味微苦。"

sent_chars = list(sentence)
sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
y_pred = model.predict(sent2id_new)
y_label = np.argmax(y_pred, axis=2)
y_label = y_label.reshape(1, -1)[0]
y_ner = [idx2label[i] for i in y_label][-len(sent_chars):]

print(idx2label)
print(sent_chars)
print(sent2id)
print(y_ner)

{0: 'O', 1: 'DISEASE', 2: 'DISEASE_GROUP', 3: 'DRUG_DOSAGE', 4: 'DRUG_EFFICACY', 5: 'DRUG_INGREDIENT', 6: 'DRUG_TASTE', 7: 'FOOD_GROUP', 8: 'PERSON_GROUP', 9: 'SYMPTOM', 10: 'SYNDROME'}
[' ', '北', '京', '同', '仁', '堂', '科', '技', '发', '展', '股', '份', '有', '限', '公', '司', '制', '药', '厂', ' ', ' ', '1', '.', '忌', '食', '辛', '辣', '，', '少', '进', '油', '腻', '。', ' ', '2', '.', '感', '冒', '发', '热', '病', '人', '不', '宜', '服', '用', '。', ' ', '3', '.', '有', '高', '血', '压', '、', '心', '脏', '病', '、', '肝', '病', '、', '糖', '尿', '病', '、', '肾', '病', '等', '慢', '性', '病', '严', '重', '者', '应', '在', '医', '师', '指', '导', '下', '服', '用', '。', ' ', '4', '.', '伴', '有', '月', '经', '紊', '乱', '者', '，', '应', '在', '医', '师', '指', '导', '下', '服', '用', '。', ' ', '5', '.', '眩', '晕', '症', '状', '较', '重', '者', '，', '应', '及', '时', '去', '医', '院', '就', '诊', '。', ' ', '6', '.', '服', '药', '2', '周', '症', '状', '无', '缓', '解', '，', '应', '去', '医', '院', '就', '诊', '。', ' ', '7', '.', '对', '本', '品', '过', '敏', '者', '禁', '用', '，', '过', '敏', '体', '质', '者'

In [11]:
list(zip(sent_chars,y_ner))

[(' ', 'O'),
 ('北', 'O'),
 ('京', 'O'),
 ('同', 'O'),
 ('仁', 'O'),
 ('堂', 'O'),
 ('科', 'O'),
 ('技', 'O'),
 ('发', 'O'),
 ('展', 'O'),
 ('股', 'O'),
 ('份', 'O'),
 ('有', 'O'),
 ('限', 'O'),
 ('公', 'O'),
 ('司', 'O'),
 ('制', 'O'),
 ('药', 'O'),
 ('厂', 'O'),
 (' ', 'O'),
 (' ', 'O'),
 ('1', 'O'),
 ('.', 'O'),
 ('忌', 'O'),
 ('食', 'O'),
 ('辛', 'O'),
 ('辣', 'O'),
 ('，', 'O'),
 ('少', 'O'),
 ('进', 'O'),
 ('油', 'O'),
 ('腻', 'O'),
 ('。', 'O'),
 (' ', 'O'),
 ('2', 'O'),
 ('.', 'O'),
 ('感', 'O'),
 ('冒', 'SYMPTOM'),
 ('发', 'O'),
 ('热', 'O'),
 ('病', 'O'),
 ('人', 'O'),
 ('不', 'O'),
 ('宜', 'O'),
 ('服', 'O'),
 ('用', 'O'),
 ('。', 'O'),
 (' ', 'O'),
 ('3', 'O'),
 ('.', 'O'),
 ('有', 'O'),
 ('高', 'O'),
 ('血', 'O'),
 ('压', 'O'),
 ('、', 'O'),
 ('心', 'O'),
 ('脏', 'O'),
 ('病', 'O'),
 ('、', 'O'),
 ('肝', 'O'),
 ('病', 'O'),
 ('、', 'O'),
 ('糖', 'O'),
 ('尿', 'O'),
 ('病', 'O'),
 ('、', 'O'),
 ('肾', 'O'),
 ('病', 'O'),
 ('等', 'O'),
 ('慢', 'O'),
 ('性', 'O'),
 ('病', 'O'),
 ('严', 'O'),
 ('重', 'O'),
 ('者', 'O'),
 ('应', 'O'),
 ('在',